In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

### Retrieve Data

In [2]:
from stockMarket.core import get_tickers_from_index

tickers = get_tickers_from_index('sp500')

In [32]:
from stockMarket.api import populate_contracts

contracts = populate_contracts(tickers)
# contracts = populate_contracts(tickers, update_period="now")
import numpy as np

from stockMarket.core import Screener, LimitScreenerObject, EqualityScreenerObject
from stockMarket.core import growth_total

screener = Screener(contracts)

bank_screener = EqualityScreenerObject('fin_statement_type', equal_to="BANK", description="Bank")
not_bank_screener = EqualityScreenerObject('fin_statement_type', not_equal_to="BANK", description="Bank")

dividend_yield_screener = LimitScreenerObject('dividend_yield', min_value=0.1, description="Dividend Yield")
payout_screener = LimitScreenerObject('payout_ratio', max_value=150, description="Payout Ratio")
price_screener = LimitScreenerObject('price', max_value=40, description="Price")
trailing_pe_screener = LimitScreenerObject('trailing_pe', min_value=0, max_value=30, description="Trailing PE")
forward_pe_screener = LimitScreenerObject('forward_pe', min_value=0, max_value=30, description="Forward PE")
revenue_screener = LimitScreenerObject('revenue', min_value=0, lambda_func= lambda x: growth_total(x)[0], description="Revenue Growth")
operating_cashflow_screener = LimitScreenerObject('operating_cashflow', min_value=0, description="Operating Cashflow")
equity_ratio_screener_not_bank = LimitScreenerObject('equity_ratio', min_value=30, description="Equity Ratio", screener_object=not_bank_screener)
equity_ratio_screener_bank = LimitScreenerObject('equity_ratio', min_value=8, description="Equity Ratio", screener_object=bank_screener)
ebitda_margin_screener = LimitScreenerObject('ebitda_margin', min_value=15, description="EBITDA Margin")
net_income_screener = LimitScreenerObject('net_income', min_value=0, lambda_func=np.mean, description="Min Net Income")

screener_list = [
    # dividend_yield_screener,
    # payout_screener,
    # price_screener,
    # trailing_pe_screener,
    # forward_pe_screener,
    # revenue_screener,
    # operating_cashflow_screener,
    # equity_ratio_screener_not_bank,
    # equity_ratio_screener_bank,
    # ebitda_margin_screener,
    # net_income_screener
]

screened_contracts = screener.screen(screener_list)

/home/jag/projects/stockMarket/stockMarket/yfinance/_common.py:36: UserWarning: Not all tickers are in calendar and the selected update period indicates that the calendar is up to date. Therefore the calendar will not be updated and the missing tickers will be remain empty. To enforce an update the calendar set update to 'now'
  warnings.warn(
/home/jag/projects/stockMarket/stockMarket/yfinance/_common.py:36: UserWarning: Not all tickers are in calendar and the selected update period indicates that the calendar is up to date. Therefore the calendar will not be updated and the missing tickers will be remain empty. To enforce an update the calendar set update to 'now'
  warnings.warn(


In [44]:
from stockMarket.core import Contract
from stockMarket.api import populate_contracts

contract = Contract(ticker='GOOG')
contracts = populate_contracts([contract])
print((contracts["GOOG"].earnings_per_share[0] - contracts["GOOG"].earnings_per_share[3])/3*100/contracts["GOOG"].earnings_per_share[3])



29.032907452275943


### Perform Screening

In [46]:
import numpy as np
import pandas as pd

from stockMarket.core import Ranking, RankingObject

def rank_equity_ratio(contract):
    equity_ratio = contract.equity_ratio[0]
    if equity_ratio > 30:
        return (equity_ratio, 1)
    if equity_ratio < 10:
        return (equity_ratio, -1)
    else:
        return (equity_ratio, 0)

def netto_margin(contract):
    netto_margin = contract.netto_margin[0]
    if 10 < netto_margin < 20:
        return netto_margin, 1
    if netto_margin > 20:
        return netto_margin, 2
    else:
        return netto_margin, 0

def return_on_assets(contract):
    return_on_assets = contract.return_on_assets[0]
    if 5 < return_on_assets < 10:
        return return_on_assets, 1
    if return_on_assets > 10:
        return return_on_assets, 2
    else:
        return return_on_assets, 0

def goodwill(contract):
    goodwill = contract.goodwill_ratio[0]
    
    if goodwill > 30 or goodwill < 0:
        return goodwill, -1
    else:    
        return goodwill, 0

def gearing(contract):
    gearing = contract.gearing[0]
    if gearing > 60:
        return gearing, -1
    if 20 < gearing < 60:
        return gearing, 0
    else:
        return gearing, 1
    
def peg_trailing_3y(contract):
    trailing_pe = contract.trailing_pe
    
    if len(contract.earnings_per_share) < 4:
        return np.nan, 0
    
    growth = contract.earnings_per_share[0] - contract.earnings_per_share[3]
    peg = contract.peg_trailing_3y
    
    if trailing_pe < 0:
        return peg, -1
    elif growth < 0:
        return peg, -1
    elif peg < 0.8:
        return peg, 1
    elif 0.8 <= peg <= 1.2:
        return peg, 0
    else:
        return peg, -1
    

equity_ratio_ranking = RankingObject(rank_equity_ratio, "Equity Ratio")
netto_margin_ranking = RankingObject(netto_margin, "Netto Margin")
return_on_assets_ranking = RankingObject(return_on_assets, "Return on Assets")
goodwill_ranking = RankingObject(goodwill, "Goodwill")
gearing_ranking = RankingObject(gearing, "Gearing")
peg_trailing_3y_ranking = RankingObject(peg_trailing_3y, "PEG Trailing 3Y")
ranking_list = [
    equity_ratio_ranking,
    netto_margin_ranking,
    return_on_assets_ranking,
    goodwill_ranking,
    gearing_ranking,
    peg_trailing_3y_ranking
]

ranking = Ranking(screened_contracts, ranking_list)
ranking.rank()
ranking.ranking.to_csv("ranking_contracts.csv", sep='\t', encoding='utf-8', float_format='%.2f')
ranking.ranking

,Name,Sector,Total Score,Equity Ratio,Equity Ratio Score,Netto Margin,Netto Margin Score,Return on Assets,Return on Assets Score,Goodwill,Goodwill Score,Gearing,Gearing Score,PEG Trailing 3Y,PEG Trailing 3Y Score
ABNB,"Airbnb, Inc.",Consumer Cyclical,7,34.667664,1,22.538397,2,11.803217,2,11.690647,0,-111.097122,1,-15.124810,1
DVN,Devon Energy Corporation,Energy,6,47.076430,1,31.378789,2,25.357278,2,6.743082,0,47.067252,0,-1.996162,1
CTRA,Coterra Energy Inc.,Energy,5,62.865932,1,44.912164,2,20.169693,2,0.000000,0,17.103394,1,12.552004,-1
WST,"West Pharmaceutical Services, Inc.",Healthcare,5,74.234130,1,20.295126,2,16.199403,2,3.996424,0,-19.598495,1,102.732745,-1
REGN,"Regeneron Pharmaceuticals, Inc.",Healthcare,5,77.577915,1,35.639823,2,14.850160,2,0.000000,0,-13.091246,1,75.660445,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MET,"MetLife, Inc.",Financial Services,-4,4.365285,-1,2.358568,0,0.229499,0,30.771281,-1,62.728636,-1,NaN,-1
NCLH,Norwegian Cruise Line Holdings Ltd.,Consumer Cyclical,-4,0.369609,-1,-46.862541,0,-12.231633,0,143.071248,-1,20400.640026,-1,NaN,-1
DVA,DaVita Inc.,Healthcare,-4,4.207912,-1,4.826917,0,3.310442,0,993.451032,-1,1447.497354,-1,-1307.662783,-1
CHTR,"Charter Communications, Inc.",Communication Services,-4,6.309722,-1,9.357299,0,3.497713,0,324.191249,-1,1162.956465,-1,11.659120,-1


### Write Email

In [47]:
from stockMarket.utils import write_email

emails = ["benjaminlantschner@gmail.com", "97gamjak@gmail.com"]
subject = "Stock Market Ranking"
body = ""
attachment = "ranking_contracts.csv"
write_email(emails, subject, body, attachment)